# Multimodal RAG for video analytics with LlamaIndex

Constructing a RAG pipeline for text is relatively straightforward, thanks to the tools developed for parsing, indexing, and retrieving text data. However, adapting RAG models for video content presents a greater challenge. Videos combine visual, auditory, and textual elements, requiring more processing power and sophisticated video pipelines.

To build a truly multimodal search for videos, you need to work with different modalities of a video like spoken content, visual. In this notebook, we showcase a Multimodal RAG pipeline designed for video analytics. It utilizes Whisper model to convert spoken content to text, CLIP model to generate multimodal embeddings, and Vision Language model (VLM) to process retrieved images and text messages. The following picture illustrates how this pipeline is working.

![Multimodal RAG](https://github.com/user-attachments/assets/baef4914-5c07-432c-9363-1a0cb5944b09)

#### Table of contents:

- [Prerequisites](#Prerequisites)
- [Convert and Compress models](#Convert-and-Compress-models)
    - [ASR model](#ASR-model)
    - [CLIP model](#CLIP-model)
    - [VLM model](#VLM-model)
- [Download and process video](#Download-and-process-video)
    - [Initialize ASR](#Initialize-ASR)
- [Create the multi-modal index](#Create-the-multi-modal-index)
    - [Initialize CLIP](#Initialize-CLIP)
- [Search text and image embeddings](#Search-text-and-image-embeddings)
- [Generate final response using VLM](#Generate-final-response-using-VLM)
    - [Set the RAG prompt template](#Set-the-RAG-prompt-template)
    - [Initialize VLM](#Initialize-VLM)
- [Interactive Demo](#Interactive-Demo)

### Installation Instructions

This is a self-contained example that relies solely on its own code.

We recommend  running the notebook in a virtual environment. You only need a Jupyter server to start.
For details, please refer to [Installation Guide](https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/README.md#-installation-guide).


<img referrerpolicy="no-referrer-when-downgrade" src="https://static.scarf.sh/a.png?x-pxid=5b5a4db0-7875-4bfb-bdbd-01698b5b1a77&file=notebooks/multimodal-rag/multimodal-rag-llamaindex.ipynb" />


## Prerequisites

[back to top ⬆️](#Table-of-contents:)

install required packages and setup helper functions.

In [1]:
%pip install -q "llama-index-core" "llama-index-embeddings-openvino>=0.4.1" "llama-index-readers-file" \
    "llama-index-vector-stores-qdrant"  \
    "transformers==4.45" \
    "pytube" \
    "moviepy==1.0.3" \
    "yt-dlp" \
    "open_clip_torch" \
    "gradio>=4.44.1" --extra-index-url https://download.pytorch.org/whl/cpu

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import requests
from pathlib import Path

os.environ["GIT_CLONE_PROTECTION_ACTIVE"] = "false"

if not Path("ov_phi3_vision_helper.py").exists():
    r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/notebooks/phi-3-vision/ov_phi3_vision_helper.py")
    open("ov_phi3_vision_helper.py", "w", encoding="utf-8").write(r.text)

if not Path("ov_phi3_llamaindex.py").exists():
    r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/notebooks/multimodal-rag/ov_phi3_llamaindex.py")
    open("ov_phi3_llamaindex.py", "w", encoding="utf-8").write(r.text)
    
if not Path("notebook_utils.py").exists():
    r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/notebook_utils.py")
    open("notebook_utils.py", "w", encoding="utf-8").write(r.text)

## Convert and Compress models

[back to top ⬆️](#Table-of-contents:)

### ASR model

[back to top ⬆️](#Table-of-contents:)

In this example, we utilize [Distil-Whisper](https://huggingface.co/distil-whisper/distil-large-v2) to recognize the spoken content in video and generate text. Distil-Whisper is a distilled variant of the [Whisper](https://huggingface.co/openai/whisper-large-v2) model by OpenAI. The Distil-Whisper is proposed in the paper [Robust Knowledge Distillation via Large-Scale Pseudo Labelling](https://arxiv.org/abs/2311.00430). According to authors, compared to Whisper, Distil-Whisper runs in several times faster with 50% fewer parameters, while performing to within 1% word error rate (WER) on out-of-distribution evaluation data.
For more information about Distil-Whisper, please refer [Distil-Whisper notebook](../distil-whisper-asr).

In [2]:
asr_model_id = "distil-whisper/distil-large-v3"
asr_model_path = asr_model_id.split("/")[-1]

if not Path(asr_model_path).exists():
    !optimum-cli export openvino --model {asr_model_id} {asr_model_path}

### CLIP model

[back to top ⬆️](#Table-of-contents:)

In this example, CLIP model will help to generate the embedding vectors for both text and images. CLIP (Contrastive Language-Image Pre-Training) is a neural network trained on various (image, text) pairs. It can be instructed in natural language to predict the most relevant text snippet, given an image, without directly optimizing for the task.

CLIP uses a [ViT](https://arxiv.org/abs/2010.11929) like transformer to get visual features and a causal language model to get the text features. The text and visual features are then projected into a latent space with identical dimensions. The dot product between the projected image and text features is then used as a similarity score.

In [3]:
clip_model_id = "laion/CLIP-ViT-B-32-laion2B-s34B-b79K"
clip_model_path = clip_model_id.split("/")[-1]

if not Path(clip_model_path).exists():
    !optimum-cli export openvino -m {clip_model_id} {clip_model_path}

### VLM model

[back to top ⬆️](#Table-of-contents:)

Vision Language model (VLM) is used to generate final response regrading the context of images and texts retrieved from vector DB. It can help to understand the both language and image instructions to complete various real-world tasks. In this example, we select [Phi-3.5-Vision](https://huggingface.co/microsoft/Phi-3.5-vision-instruct) as VLM.

The Phi-3-Vision is a lightweight, state-of-the-art open multimodal model built upon datasets which include - synthetic data and filtered publicly available websites - with a focus on very high-quality, reasoning dense data both on text and vision. The model belongs to the Phi-3 model family, and the multimodal version comes with 128K context length (in tokens) it can support. The model underwent a rigorous enhancement process, incorporating both supervised fine-tuning and direct preference optimization to ensure precise instruction adherence and robust safety measures. More details about model can be found in [model blog post](https://azure.microsoft.com/en-us/blog/new-models-added-to-the-phi-3-family-available-on-microsoft-azure/), [technical report](https://aka.ms/phi3-tech-report), [Phi-3-cookbook](https://github.com/microsoft/Phi-3CookBook)

In [4]:
from ov_phi3_vision_helper import convert_phi3_model
import nncf

vlm_model_id = "microsoft/Phi-3.5-vision-instruct"
vlm_model_path = vlm_model_id.split("/")[-1]
if not Path(vlm_model_path).exists():
    compression_configuration = {
        "mode": nncf.CompressWeightsMode.INT4_SYM,
        "group_size": 64,
        "ratio": 0.6,
        }
    convert_phi3_model(vlm_model_id, vlm_model_path, compression_configuration)

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


## Download and process video

[back to top ⬆️](#Table-of-contents:)

To begin, download an example video from YouTube and extract the audio and frame files from it.

In [5]:
video_url = "https://www.youtube.com/watch?v=d_qvLDhkg00"
output_video_path = "./video_data/"
output_folder = "./mixed_data/"
output_audio_path = "./mixed_data/output_audio.wav"

filepath = output_video_path + "input_vid.mp4"
Path(output_folder).mkdir(parents=True, exist_ok=True)

if not Path(filepath).exists():
    !yt-dlp {video_url} -f best -o {filepath}

### Initialize ASR

[back to top ⬆️](#Table-of-contents:)

Select inference device

In [6]:
from notebook_utils import device_widget

asr_device = device_widget(default="AUTO", exclude=["NPU"])

asr_device

Dropdown(description='Device:', index=2, options=('CPU', 'GPU', 'AUTO'), value='AUTO')

The Hugging Face Optimum API is a high-level API that enables us to convert and quantize models from the Hugging Face Transformers library to the OpenVINO™ IR format. For more details, refer to the [Hugging Face Optimum documentation](https://huggingface.co/docs/optimum/intel/inference).

Optimum Intel can be used to load optimized models from the [Hugging Face Hub](https://huggingface.co/docs/optimum/intel/hf.co/models) and create pipelines to run an inference with OpenVINO Runtime using Hugging Face APIs. The Optimum Inference models are API compatible with Hugging Face Transformers models.  This means we just need to replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class.

In [7]:
from optimum.intel import OVModelForSpeechSeq2Seq
from transformers import AutoProcessor, pipeline

asr_model = OVModelForSpeechSeq2Seq.from_pretrained(asr_model_path, device=asr_device.value)
asr_processor = AutoProcessor.from_pretrained(asr_model_path)

pipe = pipeline(
    "automatic-speech-recognition",
    model=asr_model,
    tokenizer=asr_processor.tokenizer,
    feature_extractor=asr_processor.feature_extractor
)

In [ ]:
from pytube import YouTube
from moviepy.video.io.VideoFileClip import VideoFileClip

def download_video(url, output_path):
    """
    Download a video from a given url and save it to the output path.

    Params:
    url (str): The url of the video to download.
    output_path (str): The path to save the video to.

    Returns:
    dict: A dictionary containing the metadata of the video.
    """
    yt = YouTube(url)
    metadata = {"Author": yt.author, "Title": yt.title, "Views": yt.views}
    yt.streams.get_highest_resolution().download(
        output_path=output_path, filename="input_vid.mp4"
    )
    return metadata


def video_to_images(video_path, output_folder):
    """
    Convert a video to a sequence of images and save them to the output folder.

    Params:
    video_path (str): The path to the video file.
    output_folder (str): The path to the folder to save the images to.

    """
    clip = VideoFileClip(video_path)
    clip.write_images_sequence(
        os.path.join(output_folder, "frame%04d.png"), fps=0.2
    )


def video_to_audio(video_path, output_audio_path):
    """
    Convert a video to audio and save it to the output path.

    Params:
    video_path (str): The path to the video file.
    output_audio_path (str): The path to save the audio to.

    """
    clip = VideoFileClip(video_path)
    audio = clip.audio
    audio.write_audiofile(output_audio_path)


def audio_to_text(audio_path):
    """
    Convert audio to text using the SpeechRecognition library.

    Params:
    audio_path (str): The path to the audio file.

    Returns:
    test (str): The text recognized from the audio.

    """
    result = pipe(audio_path, return_timestamps=True)


    return result["text"]

In this step, we will extract the images and audio from video, then convert its audio into text.

In [32]:
try:
    video_to_images(filepath, output_folder)
    video_to_audio(filepath, output_audio_path)
    text_data = audio_to_text(output_audio_path)
    
    with open(output_folder + "output_text.txt", "w") as file:
        file.write(text_data)
    print("Text data saved to file")
    file.close()
    os.remove(output_audio_path)

except Exception as e:
    raise e

Moviepy - Writing frames ./mixed_data/frame%04d.png.


Moviepy - Done writing frames ./mixed_data/frame%04d.png.
MoviePy - Writing audio in ./mixed_data/output_audio.wav


MoviePy - Done.


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


Text data saved to file


## Create the multi-modal index

[back to top ⬆️](#Table-of-contents:)

In this step, we are going to build multi-modal index and vector store to index both text and images. The CLIP model is used to generate the embedding vector for texts and images.


### Initialize CLIP

[back to top ⬆️](#Table-of-contents:)

Select inference device

In [9]:
clip_device = device_widget(default="AUTO", exclude=["NPU"])

clip_device

Dropdown(description='Device:', index=2, options=('CPU', 'GPU', 'AUTO'), value='AUTO')

Class `OpenVINOClipEmbedding` in LlamaIndex can support exporting and loading open_clip models with OpenVINO runtime. for more information, please refer [Local Embeddings with OpenVINO](https://docs.llamaindex.ai/en/stable/examples/embeddings/openvino/#openclip-model-exporter).

In [36]:
from llama_index.embeddings.huggingface_openvino import OpenVINOClipEmbedding

clip_model = OpenVINOClipEmbedding(model_id_or_path=clip_model_path, device=clip_device.value)

In [37]:
from llama_index.core.indices import MultiModalVectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import StorageContext, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader
import qdrant_client

# Create the MultiModal index
documents = SimpleDirectoryReader(output_folder).load_data()

# Create a local Qdrant vector store
client = qdrant_client.QdrantClient(":memory:")

text_store = QdrantVectorStore(
    client=client, collection_name="text_collection"
)
image_store = QdrantVectorStore(
    client=client, collection_name="image_collection"
)
storage_context = StorageContext.from_defaults(
    vector_store=text_store, image_store=image_store
)

In [38]:
Settings.embed_model = clip_model

index = MultiModalVectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    image_embed_model=Settings.embed_model,
    transformations=[SentenceSplitter(chunk_size=500, chunk_overlap=50)]
)

retriever_engine = index.as_retriever(
    similarity_top_k=2, image_similarity_top_k=5
)

## Search text and image embeddings

[back to top ⬆️](#Table-of-contents:)

To simply the prompt for VLM, we have to prepare the context of text and images regarding user's query. In this step, the most relevant context will be retrieved from vector DB through multi-modal index.

In [39]:
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.schema import ImageNode
from PIL import Image
import matplotlib.pyplot as plt
import os

def plot_images(image_paths):
    images_shown = 0
    plt.figure(figsize=(16, 9))
    for img_path in image_paths:
        if os.path.isfile(img_path):
            image = Image.open(img_path)

            plt.subplot(2, 3, images_shown + 1)
            plt.imshow(image)
            plt.xticks([])
            plt.yticks([])

            images_shown += 1
            if images_shown >= 7:
                break

def retrieve(retriever_engine, query_str):
    retrieval_results = retriever_engine.retrieve(query_str)

    retrieved_image = []
    retrieved_text = []
    for res_node in retrieval_results:
        if isinstance(res_node.node, ImageNode):
            retrieved_image.append(res_node.node.metadata["file_path"])
        else:
            display_source_node(res_node, source_length=200)
            retrieved_text.append(res_node.text)

    return retrieved_image, retrieved_text

In [40]:
query_str = "tell me more about gaussian function"

img, txt = retrieve(retriever_engine=retriever_engine, query_str=query_str)
image_documents = SimpleDirectoryReader(
    input_dir=output_folder, input_files=img
).load_data()
context_str = "".join(txt)
plot_images(img)
print(txt)

**Node ID:** 3d226fc5-6556-4f4a-9bf3-582a3de27767<br>**Similarity:** 0.6292942845978948<br>**Text:** The basic. But the question. Of the gauceau's quare, egos. Of all, why, we're a theory. for the probability, and many of the same, the same, for the theory of the real-that move to make a symmetric...<br>

**Node ID:** a293ddaf-8be1-46c9-9210-a683e3ab2eac<br>**Similarity:** 0.605394755606113<br>**Text:** ...to, that the common between. What you're a conclusion that, is the convolution. ...toe goes to the conclusion. Sogous, is a convolution. is itself. If you, a mean, and two-toe for a new. If you,...<br>

["The basic. But the question. Of the gauceau's quare, egos. Of all, why, we're a theory. for the probability, and many of the same, the same, for the theory of the real-that move to make a symmetric. I'm in a symmetric. Ande to see a number number, I'm going, and a number, and then theery to see a number. For Thee. For, I'm still, and through the most, why I'm going, I'm trying to adder. For, I'm sorry, and a certain. We Haveene-teree. We have found out of course. We're to-d, weed graph, weed-d, I'd aided. Weir and a small. We had a bit moreed. We Did That You Add-ncrap, weed-d. We Are You Are To Bears, weed-d. We Are You Added, I'd die, I'm aided. Weed-weight. Weed-weight Thameshari, a little biterer, as many different, then, the same the central. The sum. The sum, then, the sum. The sum describing, tends to the same. And then, then the sum to the sum, and the sum that's a number. But the number. The number describing the system, tends to a number. I'm that's the same thing. The sum.

## Generate final response using VLM

[back to top ⬆️](#Table-of-contents:)

### Set the RAG prompt template

[back to top ⬆️](#Table-of-contents:)

In [41]:
qa_tmpl_str = (
    "Given the provided information, including relevant images and retrieved context from the video, \
 accurately and precisely answer the query without any additional prior knowledge.\n"
    "Please ensure honesty and responsibility, refraining from any racist or sexist remarks.\n"
    "---------------------\n"
    "Context: {context_str}\n"
    "---------------------\n"
    "Query: {query_str}\n"
    "Answer: "
)

### Initialize VLM

[back to top ⬆️](#Table-of-contents:)

Select inference device

In [42]:
vlm_device = device_widget(default="AUTO", exclude=["NPU"])

vlm_device

Dropdown(description='Device:', index=2, options=('CPU', 'GPU', 'AUTO'), value='AUTO')

`OpenVINOPhi3MultiModal` class provides convenient way for running multimodal model in LlamaIndex. It accepts directory with converted model and inference device as arguments. For running model with streaming we will use `stream_complete` method.

In [43]:
from ov_phi3_llamaindex import OpenVINOPhi3MultiModal

processor = AutoProcessor.from_pretrained(
    vlm_model_path, trust_remote_code=True
)

vlm = OpenVINOPhi3MultiModal(
    model_id_or_path=vlm_model_path,
    device=vlm_device.value,
    max_new_tokens = 1024,
    generate_kwargs={"do_sample": False, "eos_token_id" : processor.tokenizer.eos_token_id},
)

In [44]:
response = vlm.stream_complete(
    prompt=qa_tmpl_str.format(
        context_str=context_str, query_str=query_str
    ),
    image_documents=image_documents,
)
for r in response:
    print(r.delta, end="")

The Gaussian function, also known as the normal distribution, is a probability distribution that is symmetric and bell-shaped. It is widely used in statistics and probability theory to model the distribution of random variables. The Gaussian function is defined by its mean and standard deviation, which determine the center and spread of the distribution. The probability density function of the Gaussian distribution is given by the equation:

f(x) = (1/√(2πσ^2)) * e^(-(x-μ)^2 / (2σ^2))

where μ is the mean and σ is the standard deviation. The Gaussian distribution is characterized by its tails approaching zero as x moves away from the mean, and its peak at the mean value. The area under the curve of the Gaussian distribution is equal to 1, representing the total probability of all possible outcomes. The Gaussian distribution is also known for its property of being fully described by its mean and standard deviation, making it a convenient and powerful tool for modeling and analyzing data

## Interactive Demo
[back to top ⬆️](#Table-of-contents:)

Now, you can try to chat with model. Upload video, provide your text message into `Input` field and click `Submit` to start communication.

In [ ]:
import gradio as gr

def build_index(video_path):
    """
    callback function for building index of vector store
    
    Params:
      video_path: path of uploaded video file
    Returns:
      vector store is ready
      
    """
    
    global retriever_engine
    progress=gr.Progress()
    progress(None, desc="Video to Images...")
    video_to_images(video_path, output_folder)
    progress(None, desc="Video to Audio...")
    video_to_audio(video_path, output_audio_path)
    progress(None, desc="Audio to Texts...")
    text_data = audio_to_text(output_audio_path)

    with open(output_folder + "output_text.txt", "w") as file:
        file.write(text_data)
    print("Text data saved to file")
    file.close()
    os.remove(output_audio_path)
    
    progress(0, desc="Building Index...")
    documents = SimpleDirectoryReader(output_folder).load_data()
    client = qdrant_client.QdrantClient(":memory:")

    text_store = QdrantVectorStore(
        client=client, collection_name="text_collection"
    )
    image_store = QdrantVectorStore(
        client=client, collection_name="image_collection"
    )
    storage_context = StorageContext.from_defaults(
        vector_store=text_store, image_store=image_store
    )
    index = MultiModalVectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        image_embed_model=Settings.embed_model,
        transformations=[SentenceSplitter(chunk_size=500, chunk_overlap=50)]
    )

    retriever_engine = index.as_retriever(
        similarity_top_k=2, image_similarity_top_k=5
    )
    return "Vector Store is Ready"
    
    
def search(history):
    """
    callback function for searching vector store

    Params:
      history: conversation history
    Returns:
      lists of retrieved images and texts
      
    """
    progress=gr.Progress()
    progress(None, desc="Searching...")
    img, txt = retrieve(retriever_engine=retriever_engine, query_str=history[-1][0])
    return img, txt 

def generate(history, images, texts)  :
    """
    callback function for running chatbot on submit button click

    Params:
      history: conversation history
      images: list of retrieved images
      texts: list of retrieved texts

    """
    progress=gr.Progress()
    progress(None, desc="Generating...")
    history[-1][1] = gr.Gallery(images)
    image_documents = SimpleDirectoryReader(
        input_dir=output_folder, input_files=images
    ).load_data()
    context_str = "".join(texts)
    
    response = vlm.stream_complete(
        prompt=qa_tmpl_str.format(
            context_str=context_str, query_str=history[-1][0]
        ),
        image_documents=image_documents,
    )
    partial_text = ""
    history = history + [[None, ""]]
    for r in response:
        partial_text += r.delta
        history[-1][1] = partial_text
        yield history


def stop():
    vlm._model.request.cancel()

In [ ]:
if not Path("gradio_helper.py").exists():
    r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/notebooks/multimodal-rag/gradio_helper.py")
    open("gradio_helper.py", "w").write(r.text)

from gradio_helper import make_demo

demo = make_demo(filepath, build_index, search, generate, stop)

try:
    demo.queue().launch()
except Exception:
    demo.queue().launch(share=True)
# if you are launching remotely, specify server_name and server_port
# demo.launch(server_name='your server name', server_port='server port in int')
# Read more in the docs: https://gradio.app/docs/